In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cta-ridership-data/CTA_-_Ridership_-_Daily_Boarding_Totals_20241108.csv
/kaggle/input/cta-ridership-data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20241108.csv


In [7]:
cta_by_st = pd.read_csv('/kaggle/input/cta-ridership-data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20241108.csv')

print(cta_by_st.columns)

Index(['station_id', 'stationname', 'date', 'daytype', 'rides'], dtype='object')


W=Weekday, A=Saturday, U=Sunday/Holiday

In [8]:
# Ensure 'date' is in datetime format
cta_by_st['date'] = pd.to_datetime(cta_by_st['date'])

# Create a 'year' column by extracting the year from the 'date' column
cta_by_st['year'] = cta_by_st['date'].dt.year

# Group by station and year, and sum the ridership ('rides')
cta_yearly = cta_by_st.groupby(['stationname', 'year'])['rides'].sum().reset_index()

# Display the first few rows of the new dataframe
print(cta_yearly.head())

  stationname  year   rides
0        18th  2001  256183
1        18th  2002  251387
2        18th  2003  251773
3        18th  2004  236536
4        18th  2005  320783


In [12]:
plt.figure(figsize=(12, 8))  # Adjust figure size
for station in stations:
    station_data = cta_yearly[cta_yearly['stationname'] == station]
    plt.plot(station_data['year'], station_data['rides'], label=station)

# Add labels and title
plt.title('Total Ridership by Year for Each Station')
plt.xlabel('Year')
plt.ylabel('Total Rides')
plt.legend(title='Station', bbox_to_anchor=(1.1, 1), loc='upper left')
plt.xticks(cta_yearly['year'].unique())
plt.tight_layout()

# Save the plot to a file
plt.savefig('ridership_by_year.png', bbox_inches='tight')  # Save with tight layout to avoid clipping

# Close the plot to free up memory
plt.close()

/tmp/ipykernel_31/1548493258.py:12: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [13]:
cta_st_info = pd.read_csv('/kaggle/input/cta-st-info/CTA_-_System_Information_-_List_of__L__Stops_20241108.csv')

print(cta_st_info.columns)

Index(['STOP_ID', 'DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
       'STATION_DESCRIPTIVE_NAME', 'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN',
       'P', 'Pexp', 'Y', 'Pnk', 'O', 'Location'],
      dtype='object')


In [17]:
print(cta_by_st.columns)
print(cta_st_info.columns)

Index(['station_id', 'stationname', 'date', 'daytype', 'rides', 'year'], dtype='object')
Index(['STOP_ID', 'DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
       'STATION_DESCRIPTIVE_NAME', 'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN',
       'P', 'Pexp', 'Y', 'Pnk', 'O', 'Location'],
      dtype='object')


In [18]:
# Rename 'stationname' in cta_by_st to 'STATION_NAME' to match cta_st_info
cta_by_st.rename(columns={'stationname': 'STATION_NAME'}, inplace=True)

# Merge the two DataFrames on 'STATION_NAME'
cta_stn_merged = pd.merge(cta_by_st, cta_st_info[['STATION_NAME', 'RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']], 
                          on='STATION_NAME', how='left')

# Check the first few rows of the merged DataFrame
print(cta_stn_merged.head())

   station_id      STATION_NAME       date daytype  rides  year    RED   BLUE  \
0       41280    Jefferson Park 2017-12-22       W   6104  2017  False   True   
1       41280    Jefferson Park 2017-12-22       W   6104  2017  False   True   
2       41000  Cermak-Chinatown 2017-12-18       W   3636  2017   True  False   
3       41000  Cermak-Chinatown 2017-12-18       W   3636  2017   True  False   
4       40280      Central-Lake 2017-12-02       A   1270  2017    NaN    NaN   

       G    BRN      P   Pexp      Y    Pnk      O  
0  False  False  False  False  False  False  False  
1  False  False  False  False  False  False  False  
2  False  False  False  False  False  False  False  
3  False  False  False  False  False  False  False  
4    NaN    NaN    NaN    NaN    NaN    NaN    NaN  


In [19]:
# Export the merged DataFrame to a CSV file
cta_stn_merged.to_csv('cta_stn_merged.csv', index=False)